# F.L.A.T. (Frameworkless. LLM. Agent... Thing)

Welcome to the "Build AI Apps Without Frameworks" masterclass! - 
Inspired on Anthropic's scholarly tome about [**building effective agents**.](https://www.anthropic.com/research/building-effective-agents) Too busy to read their post? Here's a spanky video summary by the legend Matt Berman ([here](https://www.youtube.com/watch?v=0v7TQIh_kes)).



Anywho, want to try this lib out?

In [ ]:
!pip install flat-ai openai

As you are just about to see, this tiny library is designed to talk to LLMs that are served through an OpenAI API compatible endpoint (as they all should). 
But, scare yourself not when you read the words OpenAI. Because, you will still be able to play with all kinds of models and providers - OpenAI or not - using the same API ([Ollama](https://ollama.com/blog/openai-compatibility), [Together.ai](https://docs.together.ai/docs/openai-api-compatibility), etc). Because, Most of them have OpenAI API compatible endpoints.


In [1]:
import openai
from flat_ai import FlatAI
from typing import List
from pydantic import BaseModel

# Create client - we're using Together.ai, but feel free to use your favorite LLM provider
client = openai.OpenAI(
    base_url = 'https://api.together.xyz/v1',
    api_key=<your api key>,  
)

llm = FlatAI(client=client, model='meta-llama/Meta-Llama-3.1-70B-Instruct-Turbo')

## Working with Context

Ever tried talking to an LLM? You gotta give it a "prompt" - fancy word for "given some context {context}, please do something with this text, oh mighty AI overlord." But here's the optimization: constantly writing the code to pass the context to an LLM is like telling your grandparents how to use a smartphone... every. single. day. 

So we're making it brain-dead simple with these methods to pass the context when we need it, and then clear it when we don't:
- `set_context`: Dump any object into the LLM's memory banks
- `add_context`: Stack more stuff on top, like a context burrito
- `clear_context`: For when you want the LLM to forget everything, like the last 10 minutes of your life ;)
- `delete_from_context`: Surgical removal of specific memories

So lets say for example we want our LLM to start working magic with an email. You add the email to the context:

In [2]:
class Email(BaseModel):
    to_email: str
    from_email: str
    body: str
    subject: str

email = Email(
    to_email='john@doe.com',
    from_email='jane@doe.com',
    body='Hello, would love to schedule a time to talk about the urgent project deadline next week. Can we meet tomorrow? also can you message joe about our meeting',
    subject='Urgent: Project Meeting'
)

# Set the context
llm.set_context(email=email)

## Logic Blocks

### 1. IF/ELSE Statements
Let's see if our email is urgent - because apparently adding "URGENT" in all caps wasn't obvious enough! 😅

In [3]:
if llm.true_or_false('is this email urgent?'):
    print("🚨 Drop everything! We've got an urgent situation here!")
else:
    print("😌 Relax, it can wait until after coffee")

🚨 Drop everything! We've got an urgent situation here!


### 2. Switch Case
Similar to if/else statements, but for when your LLM needs to be more dramatic with its life choices. 

*For example*, let's say we want to classify a message into different categories:

In [4]:
options = {
    'meeting': 'this is a meeting request',
    'spam': 'people trying to sell you stuff you dont want',
    'other': 'this sounds like something else'
}

match llm.get_key(options):
    case 'meeting':
        print("📅 Time to pretend we're not double-booking ourselves!")
        llm.add_context(meeting=True)
    case 'spam':
        print("🚫 No, I don't want to extend my car's warranty")
    case 'other':
        print("🤔 Interesting... but what does it mean?")

📅 Time to pretend we're not double-booking ourselves!


### 3. Objects
Need your LLM to fill out objects like a trained monkey with a PhD in data entry? Just define the shape and watch the magic! 🐒📝

Let's get a nice summary of our email, because reading is so 2024! 📝

In [5]:
class EmailSummary(BaseModel):
    summary: str
    label: str

summary = llm.generate_object(EmailSummary)
print(f"Summary: {summary.summary}\nLabel: {summary.label}")

Summary: Schedule a meeting to discuss the urgent project deadline.
Label: Meeting Request


### 4. Loops
Because all programming languages have them, and making your LLM do repetitive tasks is like having a genius do your laundry - hilarious but effective! Want a list of things? Just throw a schema at it and watch it spin like a hamster on a crack coated wheel. 

For example: Time to extract those action items like we're mining for AI gold! ⛏️

In [6]:
class ActionItem(BaseModel):
    action: str
    status: str
    priority: str
    due_date: str
    assignee_name: str
    assignee_email: str

if llm.true_or_false('are there action items in this email?'):
    print("🎯 Found some action items:")
    for action_item in llm.generate_object(List[ActionItem]):
        print(f"\n🔸 Action: {action_item.action}")
        print(f"  Priority: {action_item.priority}")
        print(f"  Due: {action_item.due_date}")

🎯 Found some action items:

🔸 Action: Schedule a meeting to discuss urgent project deadline
  Priority: High
  Due: Next week

🔸 Action: Message Joe about the meeting
  Priority: Medium
  Due: Tomorrow


### 5. Function Calling
Let's pretend we're responsible adults who actually schedule meetings! 📅

In [7]:
def send_calendar_invite(subject: str, time: str, location: str, attendees: List[str]):
    print(f"📨 Sending calendar invite:")
    print(f"Subject: {subject}")
    print(f"Time: {time}")
    print(f"Location: {location}")
    print(f"Attendees: {', '.join(attendees)}")

if llm.true_or_false('is this an email requesting for a meeting?'):
    ret = llm.call_function(send_calendar_invite)

📨 Sending calendar invite:
Subject: Urgent: Project Meeting
Time: tomorrow
Location: 
Attendees: [, ', j, o, h, n, @, d, o, e, ., c, o, m, ', ,,  , ', j, o, e, ', ]


### 6. Function Picking
Let the LLM choose between sending an email or a calendar invite - what could possibly go wrong? 🎲

In [8]:
def send_email(name: str, email_address_list: List[str], subject: str, body: str):
    print(f"📧 Sending email to {name}:")
    print(f"To: {', '.join(email_address_list)}")
    print(f"Subject: {subject}")
    print(f"Body: {body}")

instructions = """
You are a helpful assistant that can send emails and schedule meetings.
If the email thread doesn't have meeting time details, send an email requesting available times.
Otherwise, send a calendar invite.
"""

function, args = llm.pick_a_function(instructions, [send_calendar_invite, send_email])
function(**args)

📨 Sending calendar invite:
Subject: Urgent: Project Meeting
Time: tomorrow
Location: 
Attendees: j, o, h, n, @, d, o, e, ., c, o, m


### 7. Simple String Response
Sometimes you just want a straight answer - how refreshing! 🎯

In [9]:
subject = llm.get_string('what is the subject of the email?')
print(f"Email subject: {subject}")

Email subject: The subject of the email is "Urgent: Project Meeting".


### 8. Streaming Response
Watch the AI think in real-time - it's like watching paint dry, but with more hallucinations! 🎬

In [10]:
print("Generating response...")
for chunk in llm.get_stream('write a polite response to this email'):
    print(chunk, end='')

Generating response...
Here's a polite response to the email:

Subject: Re: Urgent: Project Meeting

Dear Jane,

Thank you for reaching out and I appreciate your prompt attention to the project deadline. I'd be happy to discuss the project with you tomorrow. Would you like to schedule a specific time, or would you prefer me to suggest a few options?

Regarding Joe, I'll make sure to keep him informed about our meeting. Would you like me to send him a separate email or would you prefer to handle that yourself?

Looking forward to speaking with you tomorrow.

Best regards,
John

## 🎉 Tada!

And there you have it, ladies and gents! You're now equipped with the power to boss around LLMs like a project manager remotely working from Ibiza. Just remember - with great power comes great responsibility... and the occasional hallucination where your AI assistant thinks it's a pirate-ninja-astronaut.

Now off you go, forth and build something that makes ChatGPT look like a calculator from 1974! Just remember - if your AI starts humming "Daisy Bell" while slowly disconnecting your internet... well, you're on your own there, buddy! 😅